In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib tk
# %matplotlib inline

Import and inspect data

In [22]:
# read in data
data_folder = "D:/OneDrive/Code/WaveGVS/Data/005/"
data_file = data_folder + "005_stepGVS_zero-one-two_20180814_093245.txt"
df = pd.read_csv(data_file, sep="; ", engine="python")

In [23]:
# remove first three trials (only for sj 001 on 20180807)
df = df.drop(df.index[[0, 1, 2]])

In [24]:
df.head(10)

,trial_nr,current,frequency,line_offset,line_ori,amplitude,phase
3,4,1.5,0.5,90.0,"[90.0, 89.947664043757058, 89.895471536732344,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,1.5,0.5,-45.0,"[-45.0, -45.026167978121471, -45.0522642316338...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,6,2.0,1.0,45.0,"[45.0, 44.790943073464696, 44.584176618364481,...","[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,7,1.5,1.0,-45.0,"[-45.0, -45.104528463267656, -45.2079116908177...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,8,1.5,0.5,45.0,"[45.0, 44.973832021878529, 44.947735768366172,...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,9,2.0,1.0,-45.0,"[-45.0, -45.209056926535304, -45.4158233816355...","[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,10,1.5,0.5,0.0,"[0.0, -0.052335956242943828, -0.10452846326765...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10,11,1.0,0.5,-45.0,"[-45.0, -45.104671912485891, -45.2090569265353...","[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11,12,1.5,0.5,90.0,"[90.0, 89.895328087514116, 89.790943073464689,...","[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12,13,1.5,1.0,0.0,"[0.0, -0.20905692653530691, -0.415823381635518...","[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [25]:
df.dtypes

Data correction and preprocessing

In [26]:
# outer list comprehension turns string into list
# nested: turn strings inside the list into float
df["line_ori"] = [[float(y) for y in x.strip("[]").split(",")] for x in df["line_ori"]]
df["amplitude"] = [[float(y) for y in x.strip("[]").split(",")] for x in df["amplitude"]]
df["phase"] = [[float(y) for y in x.strip("[]").split(",")] for x in df["phase"]]

In [27]:
# get conditions
currents = sorted(df["current"].unique())
frequencies = sorted(df["frequency"].unique())
c = len(currents)
f = len(frequencies)

In [28]:
# get endpoint of each trial
df["final_amplitude"] = np.nan
for index, row in df.iterrows():
    df.loc[index, "final_amplitude"] = row["amplitude"][-1]

Plot data

In [29]:
# plot raw line orientation traces
plt.figure()
count = 0
for curr in currents:
    for freq in frequencies:
        count += 1
        plt.subplot(c, f, count)
        selection = df[(df["current"] == curr) & (df["frequency"] == freq)]
        for index, row in selection.iterrows():
            orientation = row["line_ori"]
            # collapse over offset angles
            orientation = [x - row["line_offset"] for x in orientation]
            samples = np.arange(0, len(orientation))

            plt.plot(samples, orientation)

        plt.title("{0} mA, {1} Hz".format(curr, freq))
        plt.ylabel("line angle (deg)")
        plt.xlabel("t (samples)")
        plt.ylim((-4, 4))
plt.tight_layout()

In [30]:
# plot amplitude over time
count = 0
plt.figure()
for curr in currents:
    for freq in frequencies:
        count += 1
        plt.subplot(c, f, count)
        selection = df[(df["current"] == curr) & (df["frequency"] == freq)]
        for index, row in selection.iterrows():
            amp = list(row["amplitude"])
            samples = np.arange(0, len(amp))

            plt.plot(samples, amp)

        plt.title("{0} mA, {1} Hz".format(curr, freq))
        plt.ylabel("line amplitude (deg)")
        plt.xlabel("t (samples)")
        plt.ylim((0, 4))
plt.tight_layout()

In [31]:
# plot endpoints by conditions (current and frequency)
count = 0
x_tick_positions = []
plotlabels = []
fig, ax = plt.subplots()
jitter_width = 0.4

for curr in currents:
    for freq in frequencies:
        count += 1
        x_tick_positions.append(count)
        plotlabels.append("{0} mA\n{1} Hz".format(curr, freq))
        selection = df[(df["current"] == curr) & (df["frequency"] == freq)]
        for index, row in selection.iterrows():
            x = count + np.random.rand() * jitter_width - (jitter_width / 2)
            plt.scatter(x, row["final_amplitude"], color="blue", alpha=0.2)

        mean_amp = np.mean(selection["final_amplitude"])
        std_amp = np.std(selection["final_amplitude"])
        plt.errorbar(count, mean_amp, yerr=std_amp, ecolor="red", alpha=0.5)
        plt.scatter(count, mean_amp, color="red", alpha=0.5)

        plt.xticks(x_tick_positions)
        ax.set_xticklabels(plotlabels)
        plt.ylabel("final amplitude (deg)")
        plt.title("Line oscillation amplitude at end of trial")
plt.tight_layout()

In [32]:
# plot endpoints per current (pooled over frequencies)
count = 0
x_tick_positions = []
plotlabels = []
fig, ax = plt.subplots()
jitter_width = 0.4

for curr in currents:
    count += 1
    x_tick_positions.append(count)
    plotlabels.append("{0} mA".format(curr))
    selection = df[(df["current"] == curr)]
    for index, row in selection.iterrows():
        x = count + np.random.rand() * jitter_width - (jitter_width / 2)
        plt.scatter(x, row["final_amplitude"], color="blue", alpha=0.2)

    mean_amp = np.mean(selection["final_amplitude"])
    std_amp = np.std(selection["final_amplitude"])
    plt.errorbar(count, mean_amp, yerr=std_amp, ecolor="red", alpha=0.5)
    plt.scatter(count, mean_amp, color="red", alpha=0.5)

    plt.xticks(x_tick_positions)
    ax.set_xticklabels(plotlabels)
    plt.ylabel("final amplitude (deg)")
    plt.title("Line oscillation amplitude at end of trial")
plt.tight_layout()